In [11]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

In [12]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop'

## Import Dependencies

In [13]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [14]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [15]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [16]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [17]:
staffassignment_data = pd.read_sql("SELECT * FROM staffassignment", conn)
staffassignment_data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


## Rename Columns

In [18]:
staffassignment_data=staffassignment_data.rename(columns={"ID_StaffAssignment":"ID_StaffAssignment__c",
                                                          "ID_Staff":"ID_Staff__c", "ID_Class":"ID_Class__c","Role":"Name",
                                                           "StartDate":"StartDate__c","EndDate":"EndDate__c"})
#convert date to string otherwise we will get JSON nonserilizable error
staffassignment_data["StartDate__c"]=staffassignment_data["StartDate__c"].astype(str)

staffassignment_data.head()

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Name,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [ ]:
query=sf.query("SELECT ID_Staff")

## Convert DataFrame to a list of Dictionaries

In [19]:
staffassignment_load = staffassignment_data.to_dict('records')
staffassignment_load 

[{'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_StaffAssignment__c': 2,
  'ID_Staff__c': 4,
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_StaffAssignment__c': 3,
  'ID_Staff__c': 3,
  'ID_Class__c': 1,
  'Name': 'Instructor',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None}]

## Inserting rows in bulk

In [20]:
try:
    sf.bulk.StaffAssignment__c.insert(staffassignment_load)
except Exception as e:
    print(e)